In [2]:
import pandas as pd
import os
from config import RAW_DATA_DIR
from utils.theorem_extraction import file_theorem_info

In [2]:
## TODO need to remove after commmits are consistent
from data_generation.git_extraction_helper import file_commits
commit_sha = 'a261710852a957a7d20d89b962e4b59887549f21'
all_commits = file_commits()
temp = [c.hexsha for c in all_commits].index(commit_sha)
REF_COMMIT = all_commits[temp]
##

In [18]:
def remove_error_msg_redundancies(error_msg):
    """Removes the unnecessary parts of the error message."""
    msg_by_lines = error_msg.splitlines()

    for idx, line in enumerate(msg_by_lines):
        # find the index of the line starting with 'trace: '
        if line.strip().startswith('trace:'):
            start_idx = idx + 1
    
        # Find the index of the line 'Some builds logged failures:'
        if line == 'error: Lean exited with code 1':
            end_idx = idx
            break
    
    # extract lines between the two points
    extracted_lines = '\n'.join(msg_by_lines[start_idx: end_idx])
    
    return extracted_lines

def get_ground_truth_info(df):
    """
    Obtain the dictionary of the ground truth information on the theorems in the given dataset.

    This contains the correct theorem statement and proof from the reference commit.
    """
    thm_info_dict = {}
    for filepath in df['filepath'].unique():
        thm_names_ls = df['thm_name'][df['filepath'] == filepath]
        fp_dict = file_theorem_info(REF_COMMIT, filepath, thm_names_ls)
        thm_info_dict = thm_info_dict | fp_dict
    return thm_info_dict

def query_ground_truth(row, ground_truth_table):
    """
    Wrapper function to help index and determine the ground truth for a given row in the dataset.
    """
    return pd.Series(ground_truth_table[(row['filepath'], row['thm_name'])])

In [3]:
json_files = [f for f in os.listdir(RAW_DATA_DIR)]

dataframes = []
for json_file in json_files:
    file_path = os.path.join(RAW_DATA_DIR, json_file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)

# remove duplicate rows
df = df.drop_duplicates(subset=['filepath', 'thm_name', 'failed_proof', 'error_msg'])

In [19]:
json_files = [f for f in os.listdir(RAW_DATA_DIR)]

dataframes = []
for json_file in json_files:
    file_path = os.path.join(RAW_DATA_DIR, json_file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)

# remove duplicate rows
df = df.drop_duplicates(subset=['filepath', 'thm_name', 'failed_proof', 'error_msg'])

# remove redundancies in error message
df['error_msg'] = df['error_msg'].apply(remove_error_msg_redundancies)

# append the ground truth information to the dataset
ground_truth_table = get_ground_truth_info(df)
df[['statement', 'proof']] = df.apply(lambda row: query_ground_truth(row, ground_truth_table), axis=1)
# print(len(df))

# remove all empty proofs
df = df[df['failed_proof'] != '']
df = df[df['proof'] != '']

df = df.reset_index(drop=True)

In [24]:
df.to_csv('processed_dataset.csv', index=False)

In [25]:
k = pd.read_csv('processed_dataset.csv')

In [26]:
diff = k != df

# identify rows with differences
rows_with_differences = diff.any(axis=1)

# extract specific rows where differences occur
rows_diff_df1 = df[rows_with_differences]
rows_diff_df2 = k[rows_with_differences]